<a href="https://colab.research.google.com/github/01danl/01danl/blob/main/krisha_astana_full_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [117]:
#import libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [118]:
df = pd.read_csv('krisha_astana_rentals.csv')

In [119]:
df.head()

,Rooms,Area,Floor,District,Price,Address,Details
0,4,178.6 м²,2/6 этаж,Есильский р-н,90 000 000〒,"мкр Комсомольский, Умай Ана 8","жил. комплекс Comfort House, кирпичный дом, 20..."
1,3,85 м²,3/8 этаж,Есильский р-н,67 500 000〒,Мәңгілік Ел 42а,"В залоге, жил. комплекс Aq-Tay, кирпичный дом,..."
2,2,54.91 м²,6/12 этаж,Сарыарка р-н,18 120 300〒,Бейбарыс Султан б/н,"жил. комплекс Rio de Janeiro, кирпичный дом, 2..."
3,2,68 м²,9/16 этаж,Есильский р-н,41 900 000〒,Әнет баба 9/1,"жил. комплекс ONER, монолитный дом, 2022 г.п.,..."
4,2,42.1 м²,5/12 этаж,Сарыарка р-н,21 000 000〒,Чингиз Айтматов 77/10,"жил. комплекс Qaiyndy, кирпичный дом, 2025 г.п..."


In [120]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1583 entries, 0 to 1582
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Rooms     1583 non-null   int64 
 1   Area      1583 non-null   object
 2   Floor     1226 non-null   object
 3   District  1583 non-null   object
 4   Price     1583 non-null   object
 5   Address   1542 non-null   object
 6   Details   1583 non-null   object
dtypes: int64(1), object(6)
memory usage: 86.7+ KB


In [121]:
df.shape

(1583, 7)

In [122]:
df.describe()

,Rooms
count,1583.000000
mean,2.260265
std,0.956305
min,1.000000
25%,2.000000
50%,2.000000
75%,3.000000
max,8.000000


In [123]:
df.isnull().sum()

,0
Rooms,0
Area,0
Floor,357
District,0
Price,0
Address,41
Details,0


In [124]:
def remove_currency(price):
  str_price = str(price)
  return int(str_price.replace('〒', "").replace('\xa0', ""))

In [125]:
df['Price'] = df['Price'].apply(remove_currency)
print(df['Price'])

ValueError: invalid literal for int() with base 10: 'от 20392500'

In [ ]:
def clean_m(Area):
  Area_str = str(Area)
  return float(Area_str.replace('м²', ''))
df['Area'] = df['Area'].apply(clean_m)
df['Area']

In [ ]:
df.drop('Floor', axis=1, inplace=True)

In [ ]:
plt.figure(figsize=(12,5))

plt.subplot(1,3,1)
sns.boxplot(y=df['Price'])
plt.title('Price')

plt.subplot(1,3,2)
sns.boxplot(y=df['Area'])
plt.title('Area')

plt.subplot(1,3,3)
sns.boxplot(y=df['Rooms'])
plt.title('Rooms')

plt.tight_layout()
plt.show(
)
for col in ['Price', 'Area']:
    q_low = df[col].quantile(0.025)
    q_high = df[col].quantile(0.975)
    df = df[(df[col] >= q_low) & (df[col] <= q_high)]
print("Размер после удаления выбросов:", df.shape)

In [ ]:
df['Price'] = df['Price'].fillna(df['Price'].median())
df['Area'] = df['Area'].fillna(df['Area'].median())
df['Current_Floor'] = df['Current_Floor'].fillna(df['Current_Floor'].median())

In [ ]:
plt.figure(figsize=(9,6))
sns.scatterplot(x='Area', y='Price', hue='Rooms', size='Rooms', data=df)
plt.title('Price vs Area')
plt.show()

In [ ]:
p_districts = ['Есильский р-н', 'Сарыарка р-н', 'Алматы р-н', 'Нура р-н', 'Сарайшык р-н', 'р-н Байконур']
filltered_districts = df[df['District'].isin(p_districts)]

plt.figure(figsize=(12, 6))
sns.boxplot(x='District', y='Price', data=filltered_districts)
plt.xticks(rotation=45)
plt.title('Price by District')
plt.show()

In [ ]:
df_enc = pd.get_dummies(df, columns=['District'], drop_first=False)
df_enc = df_enc.drop(['Address', 'Details'], axis=1)

In [ ]:
from sklearn.model_selection import train_test_split
X = df_enc.drop('Price', axis=1)
y = df_enc['Price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
#Model (Linear Regression)
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"RMSE: {rmse}")
r2 = r2_score(y_test, y_pred)
print(f"r^2: {r2}")